# Training Code
This workbook is just for training the yolov9 and the detr models. In the legacy folder there is an old workbook which indicates how the dataset was scraped from the internet and then put into a standard Yolo format. 

In [1]:
!{os.sys.executable} -m pip3 install ultralytics

zsh:1: parse error near `-m'


In [2]:
# import packages
import os
import cv2
import numpy as np
from IPython.display import Video, display
import time
import torch


from utils.pretrained_deployment import download_images, download_images2, download_images_with_resize, download_images_full_size
from utils.display import *
from utils.make_dataset_nyc_landmarks import make_nyc_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# reload modules every 2 seconds to see changes in notebook
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
classes = {
    0: "EmpireState",
    1: "WTC",
    2: "432ParkAve",
    3: "UNBuilding",
    4: "Flatiron",
    5: "BrooklynBridge",
    6: "ChryslerBuilding",
    7: "MetlifeBuilding",
    8: "StatueOfLiberty",
    9: "30HudsonYards",
}

## Training Yolov9

In [4]:
from ultralytics import YOLO

# Build a YOLOv9c model from pretrained weight
model = YOLO('yolov9c.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)

YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs
New https://pypi.org/project/ultralytics/8.2.0 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.29 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=dataset.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train.cache... 944 images, 0 backgrounds, 0 corrupt: 100%|██████████| 944/944 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train/1_world_trade_center_nyc_Image_16.jpg: corrupt JPEG restored and saved



val: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train2
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/59 [01:41<?, ?it/s]


KeyboardInterrupt: 

## Training RT-DETR-L

In [6]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)

rt-detr-l summary: 673 layers, 32970476 parameters, 0 gradients
New https://pypi.org/project/ultralytics/8.2.0 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.10.0 CUDA:0 (Tesla T4, 15110MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=dataset.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=10
WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5,

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

AMP: checks passed ✅
train: Scanning /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/train... 944 images, 0 backgrounds, 0 corrupt: 100%|██████████| 944/944 [00:00<00:00, 96
train: New cache created: /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/train.cache
val: Scanning /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/valid... 249 images, 0 backgrounds, 0 corrupt:  83%|████████▎ | 249/300 [00:00<00:00, 1034/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
val: Scanning /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/valid... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 1055
val: New cache created: /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/valid.cache
Plotting labels to run

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/ultralytics/data/base.py", line 240, in __getitem__
    return self.transforms(self.get_image_and_label(index))
  File "/opt/conda/lib/python3.7/site-packages/ultralytics/data/augment.py", line 57, in __call__
    data = t(data)
  File "/opt/conda/lib/python3.7/site-packages/ultralytics/data/augment.py", line 57, in __call__
    data = t(data)
  File "/opt/conda/lib/pyth